In [ ]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive/')

!ln -s /content/gdrive/My\ Drive/ ./mydrive

%cd /content/gdrive/MyDrive/projekt_inzynierski

In [ ]:
!pip install sentence-transformers -q

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch

In [ ]:
df = pd.read_csv('df_kaggle_enriched_cleaned.csv')
df.sample(5)

In [ ]:
df['tags'] = df['tags'].str.replace('_', ' ')
df['tags'] = df['tags'].fillna("")
df.sample(5)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = SentenceTransformer('intfloat/multilingual-e5-base', device=device)

In [ ]:
required_cols = ['track_id', 'name', 'artist', 'tags']

In [ ]:
tags = df['tags']
tags

In [ ]:
embeddings = model.encode(
    tags.tolist(),
    batch_size=64,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
)

In [ ]:
embeddings

In [ ]:
df_embeddings = df[['track_id', 'name', 'artist', 'tags']].copy()
df_embeddings['tags_embedding'] = embeddings.tolist()
df_embeddings

In [ ]:
import numpy as np

In [ ]:
np.save('./tag_embeddings.npy', embeddings)

In [ ]:
df_embeddings.to_parquet('./df_embeddings.parquet', index=False)

In [ ]:
df['tags_list'] = df['tags'].apply(lambda x: [tag.strip().lower() for tag in x.split(',')])

In [ ]:
all_tags = [tag for tags in df['tags_list'] for tag in tags]
unique_tags = list(set(all_tags))

unique_tags = [tag for tag in unique_tags if tag]

In [ ]:
tag_texts = [f"passage: {t}" for t in unique_tags]

In [ ]:
tag_embeddings = model.encode(
    tag_texts,
    batch_size=64,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
).astype("float32")

In [ ]:
df_tag_embeddings = pd.DataFrame({
    'tag': unique_tags,
    'tag_embedding': tag_embeddings.tolist()
})

In [ ]:
np.save('./unique_tag_embeddings.npy', tag_embeddings)
df_tag_embeddings.to_parquet('./df_unique_tag_embeddings.parquet', index=False)